# CARGA DE DATOS (MATRICES DISTANCIA Y TIEMPO)

## Distancias y tiempos terrestres

In [17]:
# load archivos to dataframes
import pandas as pd
import numpy as np
import requests

# Load the depots and clients data
depots_df = pd.read_csv('Depots.csv')
clients_df = pd.read_csv('Clients.csv')

# extract coordinates
depots_coords = depots_df[['Longitude', 'Latitude']].values.tolist()
clients_coords = clients_df[['Longitude', 'Latitude']].values.tolist()

# Combine depots and clients
all_coords = depots_coords + clients_coords


# Build the coordinates string for OSRM
coords_str = ';'.join([f"{lon},{lat}" for lon, lat in all_coords])
# API URL
url = f"https://router.project-osrm.org/table/v1/driving/{coords_str}"

# Parameters
params = {
    'sources': ';'.join(map(str, range(len(all_coords)))),
    'destinations': ';'.join(map(str, range(len(all_coords)))),
    'annotations': 'duration,distance'
}

# Send the request
response = requests.get(url, params=params)

# Check for successful response
if response.status_code != 200:
    print(f"Error: {response.status_code}")
    print(response.text)
    exit()

data = response.json()

# Extract the distance matrix
distance_matrix = np.array(data['distances'])

# Extract the duration matrix
duration_matrix = np.array(data['durations'])

depot_labels = [f"D{i+1}" for i in range(len(depots_coords))]
client_labels = [f"C{i+1}" for i in range(len(clients_coords))]
all_labels = depot_labels + client_labels

# Convertir las matrices en DataFrames con etiquetas
distance_df = pd.DataFrame(distance_matrix, index=all_labels, columns=all_labels)
duration_df = pd.DataFrame(duration_matrix, index=all_labels, columns=all_labels)

# Guardar las matrices en archivos CSV
distance_df.to_csv('distance_matrix.csv')
duration_df.to_csv('duration_matrix.csv')


ConnectionError: HTTPSConnectionPool(host='router.project-osrm.org', port=443): Max retries exceeded with url: /table/v1/driving/-74.08124218159384,4.75021190869025;-74.10993358606953,4.5363832206427785;-74.03854814565923,4.792925960208614;-74.06706883098641,4.72167778077445;-74.13826337931849,4.607707046760958;-74.12400186370824,4.650463053612691;-74.09561875464892,4.621911772492814;-74.10975623736951,4.678960680833056;-74.09547235719887,4.735973062153282;-74.10991610076434,4.550640992537941;-74.10977422186991,4.664702960902753;-74.12408925943333,4.5791740634103135;-74.09893796560621,4.59795431125545;-74.07557103763986,4.687820646838871;-74.10708524062704,4.70949446000624;-74.09727965657427,4.605029068682624;-74.16464148202755,4.648463876533332;-74.12083799988112,4.662137416953968;-74.02213076607309,4.697499030379109;-74.17207549744595,4.649416884236942;-74.15615257246444,4.606310650273935;-74.09041145358674,4.557379705282216;-74.17802255204528,4.591594072172954;-74.1015410917749,4.75

## Distancias y Tiempos aereos

In [13]:
import pandas as pd
import numpy as np
from math import radians, sin, cos, sqrt, atan2

# Función para calcular la distancia haversiana
def haversine(coord1, coord2):
    # Coordenadas en radianes
    lon1, lat1 = radians(coord1[0]), radians(coord1[1])
    lon2, lat2 = radians(coord2[0]), radians(coord2[1])

    # Fórmula haversiana
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    # Radio de la Tierra en kilómetros
    R = 6371.0
    return R * c

# Cargar datos
depots_df = pd.read_csv('Depots.csv')
clients_df = pd.read_csv('Clients.csv')

# Extraer coordenadas
depots_coords = depots_df[['Longitude', 'Latitude']].values.tolist()
clients_coords = clients_df[['Longitude', 'Latitude']].values.tolist()

# Combinar coordenadas de depósitos y clientes
all_coords = depots_coords + clients_coords

# Crear la matriz de distancias haversianas
n = len(all_coords)
distance_matrix = np.zeros((n, n))

for i in range(n):
    for j in range(n):
        distance_matrix[i, j] = haversine(all_coords[i], all_coords[j])

# Etiquetas para filas y columnas
depot_labels = [f"D{i+1}" for i in range(len(depots_coords))]
client_labels = [f"C{i+1}" for i in range(len(clients_coords))]
all_labels = depot_labels + client_labels

# Convertir la matriz en un DataFrame con etiquetas
distance_df = pd.DataFrame(distance_matrix, index=all_labels, columns=all_labels)

# Guardar la matriz en un archivo CSV
distance_df.to_csv('distance_dron_matrix.csv')

print("Matriz de distancias haversianas guardada como 'distance_matrix.csv'.")


Matriz de distancias haversianas guardada como 'distance_matrix.csv'.
